In [1]:
import numpy as np
import pandas as pd
import os
import logging
import arviz as az
import scipy.stats as stats


In [2]:
# Set constant
PATH_JOINT_MODELING = '/Users/hyijie/Desktop/CPP_Multiverse/CPP_Multiverse_RR_Stage2/mid_van_Vugt_2019/results/Memory/joint_modeling/data_joint_modeling_across_subject.csv'
PATH_HDDM = '/volumes/hyijie_psy/CPP_stage2_HDDM/mid_van_2019/model_fitted/m1_v.nc'
CPP_LIST = ['ams','pams','slps','ams_bin','pam_bin','slp_bin']


In [ ]:
# Load CPP and HDDM datasets
data_joint_modeling = pd.read_csv(PATH_JOINT_MODELING)

data_joint_modeling['rt'] = data_joint_modeling['rt']/1000
data_joint_modeling.rename(columns={'acc': 'response','similarity_bin':'condition'}, inplace=True)
data_joint_modeling = data_joint_modeling.query("trial_type == 'non-match'").reset_index(drop=True)
data_joint_modeling['subj_idx'] = data_joint_modeling['subj_idx'].str.replace('ACC0', '')

data_cpp_mean = data_joint_modeling.groupby(['subj_idx', 'condition'])[CPP_LIST].mean().reset_index()
m_infdata = az.from_netcdf(PATH_HDDM)

In [4]:
# Extract posterior from HDDM
SUBJECT_IDX = range(14, 37)
CON_IDX = [f'v_subj({i:.1f}).' for i in range(1, 5)]

data_v_posterior = []

for sub in SUBJECT_IDX:
    for i, con in enumerate(CON_IDX, start=1):
        idx = f"{con}{sub}"

        # Extract posterior for per subject
        v_posterior = m_infdata.posterior[idx].stack(sample=("chain", "draw")).values

        # Set a dataframe for storage
        df = pd.DataFrame({
            'subj_idx': sub,
            'condition': i,
            'v_posterior': v_posterior
        })
        data_v_posterior.append(df)

data_v_posterior = pd.concat(data_v_posterior, ignore_index=True)
data_v_posterior_mean = data_v_posterior.groupby(['subj_idx', 'condition'])['v_posterior'].mean().reset_index()
        

NameError: name 'm_infdata' is not defined

In [5]:
data_v_posterior_mean = pd.read_csv('/Users/hyijie/Desktop/data_v_posterior.csv')
# Caluculate absolute value for posterior
data_v_posterior_mean_abs = data_v_posterior_mean.copy()
baseline = data_v_posterior_mean_abs[data_v_posterior_mean_abs['condition'] == 1].set_index('subj_idx')['v_posterior']

data_v_posterior_mean_abs['baseline'] = data_v_posterior_mean_abs['subj_idx'].map(baseline)

data_v_posterior_mean_abs.loc[data_v_posterior_mean_abs['condition'] != 1, 'v_posterior'] += data_v_posterior_mean_abs.loc[data_v_posterior_mean_abs['condition'] != 1, 'baseline']

data_v_posterior_mean_abs.drop(columns=['baseline'], inplace=True)

In [ ]:
# Combine CPP value and absolute posterior value
data_cpp_mean[['subj_idx', 'condition']] = data_cpp_mean[['subj_idx', 'condition']].astype(int)
data_v_posterior_mean_abs[['subj_idx', 'condition']] = data_v_posterior_mean_abs[['subj_idx', 'condition']].astype(int)

data4two_steps = pd.merge(data_cpp_mean, data_v_posterior_mean_abs, on=['subj_idx','condition'] , how='inner')
#data4two_steps.to_csv('/Users/hyijie/Desktop/data4two_steps.csv')
